Connect to google drive. Google drive contains the data folder downloaded from kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# TODO: Fill in the Google Drive path where you uploaded assignment1
# Example: If you create a Fall2023 folder and put all the files under A1 folder, then 'Fall2023/A1'
GOOGLE_DRIVE_PATH_POST_MYDRIVE = 'assignment3_NLP_sum25'
GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'MyDrive', GOOGLE_DRIVE_PATH_POST_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['Machine_Translation.ipynb', 'trans_model.png', 'assignment3_nlp_submission.zip', '~$assignment3-nlp-template.pptx', 'seq2seq_model_RNN.png', 'seq2seq_model_LSTM.png', 'seq2seq_model_LSTM_attn.png', 'seq2seq_model_RNN_attn.png', 'seq2seq_model.png', 'models', '__pycache__', 'imgs', 'data', 'environment.yml', 'assignment3_NLP_sum25.pdf', 'requirements.txt', 'utils.py', 'assignment3-nlp-template.pptx', 'collect_submission.sh', 'collect_submission.bat']


In [ ]:
import sys

sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
# if running locally set GOOGLE PATH
import sys
if 'google.colab' in sys.modules:
  print(f'Running in google colab. Our path is `{GOOGLE_DRIVE_PATH}`')
else:
  GOOGLE_DRIVE_PATH = '.'
  print('Running locally.')

Running in google colab. Our path is `/content/drive/MyDrive/assignment3_NLP_sum25`


In [ ]:
# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from collections import Counter
from torch.nn.utils.rnn import pad_sequence

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation
from utils import train, evaluate, set_seed_nb, unit_test_values, deterministic_init, plot_curves

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Now lets check your GPU availability and load some sanity checkers. By default you should be using your gpu for this assignment if you have one available.

In [ ]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cpu
